# **Fine-tuning Wav2Vec2 for English ASR with 🤗 Transformers**

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')


[source](https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_tuning_Wav2Vec2_for_English_ASR.ipynb#scrollTo=WPq-vSxfT4w8)

Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.

Using a novel contrastive pretraining objective, Wav2Vec2 learns powerful speech representations from more than 50.000 hours of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/wav2vec2.png)

For the first time, it has been shown that pretraining, followed by fine-tuning on very little labeled speech data achieves competitive results to state-of-the-art ASR systems. Using as little as 10 minutes of labeled data, Wav2Vec2 yields a word error rate (WER) of less than 5% on the clean test set of [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) - *cf.* with Table 9 of the [paper](https://arxiv.org/pdf/2006.11477.pdf).

In this notebook, we will give an in-detail explanation of how Wav2Vec2's pretrained checkpoints can be fine-tuned on any English ASR dataset. Note that in this notebook, we will fine-tune Wav2Vec2 without making use of a language model. It is much simpler to use Wav2Vec2 without a language model as an end-to-end ASR system and it has been shown that a standalone Wav2Vec2 acoustic model achieves impressive results. For demonstration purposes, we fine-tune the "base"-sized [pretrained checkpoint](https://huggingface.co/facebook/wav2vec2-base) on the rather small [Timit](https://huggingface.co/datasets/timit_asr) dataset that contains just 5h of training data.

Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition. 

I highly recommend reading the blog post [Sequence Modeling with CTC (2017)](https://distill.pub/2017/ctc/) very well-written blog post by Awni Hannun.

First, let's try to get a good GPU in our colab! With Google Colab's free version it's sadly becoming much harder to get access to a good GPU. With Google Colab Pro, one has a much easier time getting access to a V100 or P100 GPU however.

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri May  6 14:12:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000001:00:00.0 Off |                    0 |
| N/A   38C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           On   | 00000002:00:00.0 Off |                    0 |
| N/A   

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [6]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

Next we strongly suggest to upload your training checkpoints directly to the [🤗 Hub](https://huggingface.co/) while training. The [🤗 Hub](https://huggingface.co/) has integrated version control so you can be sure that no model checkpoint is getting lost during training. 

To do so you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

#!huggingface-cli login


Then you need to install Git-LFS to upload your model checkpoints:

In [7]:
%%capture
!apt install git-lfs




---

${}^1$ Timit is usually evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). To keep this notebook as general as possible we decided to evaluate the model using WER.

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text. 

In 🤗 Transformers, the Wav2Vec2 model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).

Let's start by creating the tokenizer responsible for decoding the model's predictions.

### Create Wav2Vec2CTCTokenizer

The [pretrained Wav2Vec2 checkpoint]( ) maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *"BERT"* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Timit and define a vocabulary based on the dataset's transcriptions.

Let's start by loading the dataset and taking a look at its structure.

https://huggingface.co/docs/datasets/audio_process



https://discuss.huggingface.co/t/json-dump-format-for-load-dataset/9974

In [8]:
%%capture
%pip install datasets[audio]

Many ASR datasets only provide the target text, `'text'` for each audio `'audio'` and file `'file'`. Timit actually provides much more information about each audio file, such as the `'phonetic_detail'`, etc., which is why many researchers choose to evaluate their models on phoneme classification instead of speech recognition when working with Timit. However, we want to keep the notebook as general as possible, so that we will only consider the transcribed text for fine-tuning.



In [10]:
import os
os.listdir()

['.amlignore',
 '.amlignore.amltmp',
 '.ipynb_aml_checkpoints',
 '1gpu-westeu.out',
 'asr-1gpu-westeu-copy.out',
 'assembly_api_az_mount.ipynb',
 'az_mount_analysis.ipynb',
 'connection.cfg',
 'data',
 'data.csv',
 'finetune_wav2vec.py',
 'finetune_wav2vec.py.amltmp',
 'finetune_wav2vec2_hf.ipynb',
 'finetune_wav2vec2_hf.ipynb.amltmp',
 'finetune_wav2vec2_hf_.ipynb',
 'finetune_wav2vec2_hf_.ipynb.amltmp',
 'finetune_wav2vec2_hf_old.ipynb',
 'finetune_wav2vec_wandb-copy.py',
 'finetune_wav2vec_wandb-copy.py.amltmp',
 'finetune_wav2vec_wandb.py',
 'finetune_wav2vec_wandb.py.amltmp',
 'finetuning-wandb.ipynb',
 'finetuning-wandb.ipynb.amltmp',
 'finetuning.ipynb',
 'finetuning.ipynb.amltmp',
 'finetuning_nvidia.ipynb',
 'finetuning_nvidia.ipynb.amltmp',
 'Fine_tuning_Wav2Vec2_for_English_ASR.ipynb',
 'hparam.yml',
 'hparam.yml.amltmp',
 'hparams.yaml',
 'hparams_dl.yaml',
 'hparams_ref.yaml',
 'inference.ipynb',
 'inference.ipynb.amltmp',
 'make_dataset_az_mnt.ipynb',
 'mount_blob.py',
 '

In [ ]:
from datasets import load_dataset, load_metric, Audio

data = load_dataset('json', data_files='tcrs_90s_datacruch.json')
dataset = tcrs['train'].cast_column("audio", Audio(sampling_rate=16000)

# checking that mapping worked correctly
dataset[-1]['audio']

In [11]:
datasets = data.train_test_split(0.1)


NameError: name 'data' is not defined

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [16]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(datasets['train'], num_examples=1)

Alright! The transcriptions look very clean and the language seems to correspond more to written text than dialogue. This makes sense taking into account that [Timit](https://huggingface.co/datasets/timit_asr) is a read speech corpus.

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [12]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

datasets = datasets.map(remove_special_characters)

NameError: name 'dataset' is not defined

In [19]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs =datasets.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names) 
vocab_list = list(set(vocabs['train']["vocab"][0]) | set(vocabs['test']["vocab"][0]))

### our vocab dict has numbers & currency --> probably need to find and replace numbers to words
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

## setting up token mapping for tokenizer
# remapping space token
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

# adding unk, pad tokenns
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)


import json
    
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


In [25]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
repo_name = "wav2vec2-tcrs"
tokenizer.push_to_hub(repo_name)

### Create Wav2Vec2 Feature Extractor

In [28]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, ampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [29]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [14]:
import IPython.display as ipd
import numpy as np
import random

#rand_int = random.randint(0, len(train))

print(train[rand_int]["text"])
#ipd.Audio(data=np.asarray(train[rand_int]["audio"]["array"]), autoplay=True, rate=16000)

#rand_int = random.randint(0, len(timit["train"]))

#print("Target text:", timit["train"][rand_int]["text"])
#print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
#print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])

NameError: name 'train' is not defined

https://huggingface.co/docs/datasets/audio_process

In [8]:
#%pip install datasets[audio]
#!sudo apt-get install libsndfile1

In [30]:
import torchaudio
import numpy as np

def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
 
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

datasets = datasets.map(prepare_dataset,num_proc=4)

Let's apply the data preparation function to all examples.

**Note**: Currently `datasets` make use of [`torchaudio`](https://pytorch.org/audio/stable/index.html) and [`librosa`](https://librosa.org/doc/latest/index.html) for audio loading and resampling. If you wish to implement your own costumized data loading/sampling, feel free to just make use of the `"path"` column instead and disregard the `"audio"` column.

Long input sequences require a lot of memory. Since `Wav2Vec2` is based on `self-attention` the memory requirement scales quadratically with the input length for long input sequences (*cf.* with [this](https://www.reddit.com/r/MachineLearning/comments/genjvb/d_why_is_the_maximum_input_sequence_length_of/) reddit post). For this demo, let's filter all sequences that are longer than 4 seconds out of the training dataset.

Awesome, now we are ready to start training!

## Training & Evaluation

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [32]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)±

In [35]:
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [36]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(vocab_dict)
)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.weight', 'project_hid.bias', 'project_hid.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_q.bias', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a mo

The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the [paper](https://arxiv.org/abs/2006.11477) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

During training, a checkpoint will be uploaded asynchronously to the hub every 400 training steps. It allows you to also play around with the demo widget even while your model is still training.

**Note**: If one does not want to upload the model checkpoints to the hub, simply set `push_to_hub=False`.

In [41]:
from transformers import TrainingArguments


training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=1,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=10,
  eval_steps=10,
  logging_steps=10,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=10,
  save_total_limit=2,
  push_to_hub=True,
  report_to="wandb",  # enable logging to W&B
  run_name="wav2vec2_hf_test"  # name of the W&B run (optional)
)

PyTorch: setting up devices


Now, all instances can be passed to Trainer and we are ready to start training!

In [42]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=valid,
    tokenizer=processor.feature_extractor,
)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/asr-westeu-2gpu/code/Users/neelan/asr/pipeline/wav2vec2-tcrs is already a clone of https://huggingface.co/neelan-elucidate-ai/wav2vec2-tcrs. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp fp16 backend


### Training

In [43]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: text, sample_rate, audio, num_channels, words_per_second, num_frames, bits_per_sample, spkID, encoding, input_length, word_count, file, duration_seconds.
***** Running training *****
  Num examples = 12
  Num Epochs = 30
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 180
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py", line 1529, in forward
    loss = nn.functional.ctc_loss(
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/nn/functional.py", line 2599, in ctc_loss
    return torch.ctc_loss(
RuntimeError: blank must be in label range


The final WER should be around 0.3 which is reasonable given that state-of-the-art phoneme error rates (PER) are just below 0.1 (see [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit)) and that WER is usually worse than PER.

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-TCRS-test
Configuration saved in wav2vec2-TCRS-test/config.json
Model weights saved in wav2vec2-TCRS-test/pytorch_model.bin
Configuration saved in wav2vec2-TCRS-test/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 3.34k/360M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 2.80k/2.80k [00:00<?, ?B/s]

KeyboardInterrupt: ignored

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier "your-username/the-name-you-picked" so for instance:

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-colab")
```

### Evaluate

In the final part, we run our model on some of the validation data to get a feeling for how well it works.

Let's load the `processor` and `model`.

In [95]:
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-colab")

https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/preprocessor_config.json not found in cache or force_download set to True, downloading to /home/azureuser/.cache/huggingface/transformers/tmpwo_ifqx5


Downloading:   0%|          | 0.00/215 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/preprocessor_config.json in cache at /home/azureuser/.cache/huggingface/transformers/6650b23ff063cfae42c42902d0ba3b51e63194f251f97553e8e28353efc63dd6.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
creating metadata file for /home/azureuser/.cache/huggingface/transformers/6650b23ff063cfae42c42902d0ba3b51e63194f251f97553e8e28353efc63dd6.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
loading feature extractor configuration file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/preprocessor_config.json from cache at /home/azureuser/.cache/huggingface/transformers/6650b23ff063cfae42c42902d0ba3b51e63194f251f97553e8e28353efc63dd6.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size":

Downloading:   0%|          | 0.00/268 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/vocab.json in cache at /home/azureuser/.cache/huggingface/transformers/e6d290d17f47887c0d335cde9e4c75b2b8be89150da3f448962299852d0c1b90.c88805603b1aa90845c323683fc4b63d2d0a3797d45c3be1869fb9d0b518907f
creating metadata file for /home/azureuser/.cache/huggingface/transformers/e6d290d17f47887c0d335cde9e4c75b2b8be89150da3f448962299852d0c1b90.c88805603b1aa90845c323683fc4b63d2d0a3797d45c3be1869fb9d0b518907f
https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /home/azureuser/.cache/huggingface/transformers/tmph7vnwx8u


Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/tokenizer_config.json in cache at /home/azureuser/.cache/huggingface/transformers/cd7fc4d8728f99fa8338983a6f39ff72bc2a8a998c96b495cc13ea7aa2ececc4.9a832b91e3d317e7bec01bab5134265b75f2d6b05a270d0140d8a116c51b8591
creating metadata file for /home/azureuser/.cache/huggingface/transformers/cd7fc4d8728f99fa8338983a6f39ff72bc2a8a998c96b495cc13ea7aa2ececc4.9a832b91e3d317e7bec01bab5134265b75f2d6b05a270d0140d8a116c51b8591
https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /home/azureuser/.cache/huggingface/transformers/tmpqt9tlxlf


Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/special_tokens_map.json in cache at /home/azureuser/.cache/huggingface/transformers/00f71aa52e24e9186af116542c036dec7227ff5de6fa0be433c98afe2a82c547.a21d51735cf8667bcd610f057e88548d5d6a381401f6b4501a8bc6c1a9dc8498
creating metadata file for /home/azureuser/.cache/huggingface/transformers/00f71aa52e24e9186af116542c036dec7227ff5de6fa0be433c98afe2a82c547.a21d51735cf8667bcd610f057e88548d5d6a381401f6b4501a8bc6c1a9dc8498
loading file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/vocab.json from cache at /home/azureuser/.cache/huggingface/transformers/e6d290d17f47887c0d335cde9e4c75b2b8be89150da3f448962299852d0c1b90.c88805603b1aa90845c323683fc4b63d2d0a3797d45c3be1869fb9d0b518907f
loading file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/tokenizer_config.json from cache at /home/azureuser/.cache/huggingface/transformers/cd7fc4d8

In [96]:
model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-colab")

https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/config.json not found in cache or force_download set to True, downloading to /home/azureuser/.cache/huggingface/transformers/tmp91xic60u


Downloading:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/config.json in cache at /home/azureuser/.cache/huggingface/transformers/490d8f8aeaf5d4966ce3ade581eef540822500b93e043d04cb298fe305b2de26.1578a87effe8421b6d0f1bf16300b173f770d1f2401836d19115dbc545b00fde
creating metadata file for /home/azureuser/.cache/huggingface/transformers/490d8f8aeaf5d4966ce3ade581eef540822500b93e043d04cb298fe305b2de26.1578a87effe8421b6d0f1bf16300b173f770d1f2401836d19115dbc545b00fde
loading configuration file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/config.json from cache at /home/azureuser/.cache/huggingface/transformers/490d8f8aeaf5d4966ce3ade581eef540822500b93e043d04cb298fe305b2de26.1578a87effe8421b6d0f1bf16300b173f770d1f2401836d19115dbc545b00fde
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "at

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/pytorch_model.bin in cache at /home/azureuser/.cache/huggingface/transformers/826b895a762fbbb2294ba6b06628bb5e1ffee8e1f8cca049864f430fd5e16c16.06567df5bd322c1335ecf865b0753a748c9b0cf62496e2fc0859e937cc81508f
creating metadata file for /home/azureuser/.cache/huggingface/transformers/826b895a762fbbb2294ba6b06628bb5e1ffee8e1f8cca049864f430fd5e16c16.06567df5bd322c1335ecf865b0753a748c9b0cf62496e2fc0859e937cc81508f
loading weights file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-colab/resolve/main/pytorch_model.bin from cache at /home/azureuser/.cache/huggingface/transformers/826b895a762fbbb2294ba6b06628bb5e1ffee8e1f8cca049864f430fd5e16c16.06567df5bd322c1335ecf865b0753a748c9b0cf62496e2fc0859e937cc81508f
All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

All the weights of Wav2Vec2ForCTC were initialized from the model checkpoint at patrickvonplaten/wav2vec2-

Now, we will make use of the `map(...)` function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary `"results"`. 

**Note**: we evaluate the test data set with `batch_size=1` on purpose due to this [issue](https://github.com/pytorch/fairseq/issues/3227). Since padded inputs don't yield the exact same output as non-padded inputs, a better WER can be achieved by not padding the input at all.

In [108]:
model.config.vocab_size = len(vocabs['vocab'])

In [109]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [110]:
results = dataset.map(map_to_result, remove_columns=dataset.column_names)

0ex [00:00, ?ex/s]

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

Let's compute the overall WER now.

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.221


22.1% WER - not bad! Our demo model would have probably made it on the official [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit).

Let's take a look at some predictions to see what errors are made by the model.

In [ ]:
show_random_elements(results)

,pred_str,text
0,ame to ballenge or employ you benefit package,aim to balance your employee benefit package
1,the fawg prevented them from ariving on tome,the fog prevented them from arriving on time
2,young children should avoide exposure to contageous diseases,young children should avoid exposure to contagious diseases
3,art official intelligence is for wrheal,artificial intelligence is for real
4,their propes were two step ladders a chare and a pome fan,their props were two stepladders a chair and a palm fan
5,if people were more generous there would be no need for wealfar,if people were more generous there would be no need for welfare
6,the fish began to leep frantically on the surface of the small eack,the fish began to leap frantically on the surface of the small lake
7,her rite hand ags wheanever the barametric pressur changes,her right hand aches whenever the barometric pressure changes
8,only lawyers loved milianears,only lawyers love millionaires
9,the nearest sinnigade may not be within walk in distance,the nearest synagogue may not be within walking distance


It becomes clear that the predicted transcriptions are acoustically very similar to the target transcriptions, but often contain spelling or grammatical errors. This shouldn't be very surprising though given that we purely rely on Wav2Vec2 without making use of a language model.

Finally, to better understand how CTC works, it is worth taking a deeper look at the exact output of the model. Let's run the first test sample through the model, take the predicted ids and convert them to their corresponding tokens.

In [112]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(dataset[:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

"[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] c n [PAD] [PAD] [PAD] [PAD] ' ' [PAD] x x g g [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] i i i h h [PAD] c n i i [PAD] ' ' n n n g g g [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] c c x x g g g [PAD] [PAD] i i i i [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

The output should make it a bit clearer how CTC works in practice. The model is to some extent invariant to speaking rate since it has learned to either just repeat the same token in case the speech chunk to be classified still corresponds to the same token. This makes CTC a very powerful algorithm for speech recognition since the speech file's transcription is often very much independent of its length.

I again advise the reader to take a look at [this](https://distill.pub/2017/ctc) very nice blog post to better understand CTC.